# Data Cleaning

### Import all of the applicable librarys

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

### Combine all of the dataset, this step can be condensed if our system can download the full dataset

In [2]:
a= pd.read_csv('aug01tojan01.csv')
b = pd.read_csv('jan02tomar01.csv')
c = pd.read_csv('mar02toapr01.csv')
d = pd.read_csv('apr02toapr30.csv')
present = pd.read_csv('may01tomay18.csv')
dfcd1 = pd.concat([a,b,c,d],axis=0)

In [3]:
dfcd1['Date'] =pd.to_datetime(dfcd1.Date)
dfcd1=dfcd1.sort_values(by='Date')

In [4]:
dfcd1.to_csv('comaug01_april30.csv')

In [5]:
dfcd2 = pd.concat([dfcd1,present],axis=0)

In [6]:
dfcd2['Date'] =pd.to_datetime(dfcd2.Date)
dfcd2=dfcd2.sort_values(by='Date')

In [7]:
df=dfcd2.copy()

In [8]:
df.tail()

,Transaction Id,Reference Number,Receiving Status,Source Transaction,Date,Time,Status,Merchant Id,Merchant Name,Partner Id,...,Tip Amount,Location,Point Of Sale,Clerk User Id,Clerk External Id,Clerk First Name,Clerk Last Name,Customer User Id,Customer Email Address,Customer Phone Number
17,14490,NaN,Not Set,-1,2020-05-18,16:00:30,Approved,3702,"Grass Roots, Inc.",0,...,0.0,Grass Roots,Delivery,4956,NaN,Paul,Prusiensky,12972,leveailmyles@gmail.com,6263530441
16,14491,NaN,Not Set,-1,2020-05-18,16:00:49,Approved,2429,Earth's Healing North,2275,...,0.0,Earth's Healing North,POS 1,3824,NaN,Marquan,Graham,8848,daniel013117@gmail.com,5204998933
15,14492,NaN,Not Set,-1,2020-05-18,16:03:28,Approved,2429,Earth's Healing North,2275,...,0.0,Earth's Healing North,POS 1,3824,NaN,Marquan,Graham,3874,jaygraves39@gmail.com,5202588768
13,14494,NaN,Not Set,-1,2020-05-18,16:09:12,Approved,2430,Earths Healing South,2275,...,0.0,Earth's Healing South,POS 1,3797,NaN,Nate,Haskins,9499,cmorales816@outlook.com,5203607389
80,14427,NaN,Not Set,-1,2020-05-18,12:43:06,Approved,3702,"Grass Roots, Inc.",0,...,0.0,Grass Roots,Delivery,4956,NaN,Paul,Prusiensky,8838,emcarlson2@usfca.edu,3039052191


In [9]:
df.columns

Index(['Transaction Id', 'Reference Number', 'Receiving Status',
       'Source Transaction', 'Date', 'Time', 'Status', 'Merchant Id',
       'Merchant Name', 'Partner Id', 'Partner Name',
       'Total Amount (Including Tip)', 'Fees', 'Net Amount (Including Tip)',
       'Tip Amount', 'Location', 'Point Of Sale', 'Clerk User Id',
       'Clerk External Id', 'Clerk First Name', 'Clerk Last Name',
       'Customer User Id', 'Customer Email Address', 'Customer Phone Number'],
      dtype='object')

### Drop all non-applicable records

In [10]:
#in case there are duplicated transations when combining 
df=df.drop_duplicates(subset='Transaction Id')

In [11]:
#drop all alt36 emails
df=df[~df['Customer Email Address'].str.match('^[A-Za-z0-9._%+-]+@alt36.com$')]

In [12]:
#only keep the approved transaction
df=df[df['Status']=='Approved']

In [13]:
#only keep the transactions larger than $10
df=df[df['Total Amount (Including Tip)']>=10]

# Data wrangling

### Calculate each customer's total number of transactions 

In [14]:
df['Total_Transaction_count']=df.groupby('Customer Email Address')['Customer Email Address'].transform('size')

### Calculate each customer's total dollar amount of transactions

In [15]:
total = df[['Customer Email Address','Net Amount (Including Tip)']]
total=total.groupby(['Customer Email Address']).sum().reset_index()
total.columns=['Customer Email Address','Total_Transaction_amount']
df=df.merge(total,on='Customer Email Address',how='left')

### Extract each customer's last transaction date & calculate "Recency"

In [16]:
max_purchase = df.groupby('Customer User Id').Date.max().reset_index()
max_purchase.columns=['Customer User Id','MaxPurchaseDate']
max_purchase['MaxPurchaseDate']=pd.to_datetime(max_purchase['MaxPurchaseDate'])

In [17]:
df['Date'] = pd.to_datetime(df['Date'])
Max_date = df['Date'].max()

In [18]:
max_purchase['Recency']=(Max_date - max_purchase['MaxPurchaseDate'])
max_purchase['Recency']=max_purchase['Recency'].astype('timedelta64[D]')

In [19]:
df=df.merge(max_purchase,on='Customer User Id',how='left')

In [20]:
df=df.drop_duplicates(subset='Customer User Id')

### Calcualte "Frequency" & "Monetary"

In [21]:
rfm=df[['Customer User Id','Total_Transaction_count','Total_Transaction_amount','Recency']]

In [22]:
rfm['Recency']=rfm['Recency'].astype(int)
rfm=rfm.set_index('Customer User Id')
rfm.columns=['Frequency','Monetary','Recency']

/Users/shellydu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Break Recency, Frequency and Monetary into quantiles
### Assign scores from 1-4 for each criteria based on quantile

In [23]:
quantiles = rfm.quantile(q=[0.25,0.5,0.75])

In [24]:
RFM_Segment = rfm.copy()
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def R_Class(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FM_Class(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
RFM_Segment['R_Quartile'] = RFM_Segment['Recency'].apply(R_Class, args=('Recency',quantiles,))
RFM_Segment['F_Quartile'] = RFM_Segment['Frequency'].apply(FM_Class, args=('Frequency',quantiles,))
RFM_Segment['M_Quartile'] = RFM_Segment['Monetary'].apply(FM_Class, args=('Monetary',quantiles,))
RFM_Segment['RFMClass'] = RFM_Segment.R_Quartile.map(str) \
                            + RFM_Segment.F_Quartile.map(str) \
                            + RFM_Segment.M_Quartile.map(str)

### Combine all three criterias and generate a total RFM score

In [25]:
RFM_Segment['RFM_Score'] = RFM_Segment[['R_Quartile','F_Quartile','M_Quartile']].sum(axis=1)
print(RFM_Segment['RFM_Score'].head())

Customer User Id
2091    5
2093    9
2094    6
2095    3
2096    3
Name: RFM_Score, dtype: int64


In [26]:
df1=df.merge(RFM_Segment,on='Customer User Id',how='left')

In [27]:
df1.head()

,Transaction Id,Reference Number,Receiving Status,Source Transaction,Date,Time,Status,Merchant Id,Merchant Name,Partner Id,...,MaxPurchaseDate,Recency_x,Frequency,Monetary,Recency_y,R_Quartile,F_Quartile,M_Quartile,RFMClass,RFM_Score
0,1218,NaN,Not Set,-1,2019-08-30,14:04:33,Approved,1039,"Releaf Herbal Cooperative, INC",1,...,2019-09-13,248.0,2,64.2720,248,1,3,1,131,5
1,1231,NaN,Not Set,-1,2019-08-30,17:17:10,Approved,1039,"Releaf Herbal Cooperative, INC",1,...,2019-12-13,157.0,28,974.8992,157,1,4,4,144,9
2,1234,NaN,Not Set,-1,2019-08-30,18:32:57,Approved,1039,"Releaf Herbal Cooperative, INC",1,...,2019-09-04,257.0,2,117.0624,257,1,3,2,132,6
3,1235,NaN,Not Set,-1,2019-08-30,19:01:53,Approved,1039,"Releaf Herbal Cooperative, INC",1,...,2019-08-30,262.0,1,44.4864,262,1,1,1,111,3
4,1238,NaN,Not Set,-1,2019-08-30,20:24:16,Approved,1039,"Releaf Herbal Cooperative, INC",1,...,2019-08-30,262.0,1,20.6688,262,1,1,1,111,3


In [28]:
df1.columns

Index(['Transaction Id', 'Reference Number', 'Receiving Status',
       'Source Transaction', 'Date', 'Time', 'Status', 'Merchant Id',
       'Merchant Name', 'Partner Id', 'Partner Name',
       'Total Amount (Including Tip)', 'Fees', 'Net Amount (Including Tip)',
       'Tip Amount', 'Location', 'Point Of Sale', 'Clerk User Id',
       'Clerk External Id', 'Clerk First Name', 'Clerk Last Name',
       'Customer User Id', 'Customer Email Address', 'Customer Phone Number',
       'Total_Transaction_count', 'Total_Transaction_amount',
       'MaxPurchaseDate', 'Recency_x', 'Frequency', 'Monetary', 'Recency_y',
       'R_Quartile', 'F_Quartile', 'M_Quartile', 'RFMClass', 'RFM_Score'],
      dtype='object')

In [29]:
df1.drop(['Status','Transaction Id', 'Reference Number', 
       'Source Transaction', 'Date', 'Time','Partner Id', 'Partner Name','Location', 
        'Point Of Sale', 'Clerk User Id',
       'Clerk External Id', 'Clerk First Name', 'Clerk Last Name','MaxPurchaseDate', 
        'Recency_x', 'Frequency', 'Monetary', 'Recency_y',
       'R_Quartile', 'F_Quartile', 'M_Quartile', 'RFMClass',
        'Total Amount (Including Tip)', 'Fees', 
       'Tip Amount'],axis=1,inplace=True)

In [30]:
df1.head()

,Receiving Status,Merchant Id,Merchant Name,Net Amount (Including Tip),Customer User Id,Customer Email Address,Customer Phone Number,Total_Transaction_count,Total_Transaction_amount,RFM_Score
0,Not Set,1039,"Releaf Herbal Cooperative, INC",29.5200,2091,keeyasedwards@gmail.com,4154109717,2,64.2720,5
1,Not Set,1039,"Releaf Herbal Cooperative, INC",27.5904,2093,ludari89@gmail.com,4157163647,28,974.8992,9
2,Not Set,1039,"Releaf Herbal Cooperative, INC",54.3168,2094,jpjocelynparra@gmail.com,4159966592,2,117.0624,6
3,Not Set,1039,"Releaf Herbal Cooperative, INC",44.4864,2095,yasminmahmoudian23@gmail.com,6506192470,1,44.4864,3
4,Not Set,1039,"Releaf Herbal Cooperative, INC",20.6688,2096,msomach45@gmail.com,8584426873,1,20.6688,3


#### RFM Scores Range 3-5-7-9-12

In [31]:
df1['RFM_Score'].describe()

count    3202.000000
mean        7.033417
std         2.733927
min         3.000000
25%         5.000000
50%         7.000000
75%         9.000000
max        12.000000
Name: RFM_Score, dtype: float64

## Rejected Transactions Analysis
### Both rejected and pending-rerun transaction status are considered as "rejected transaction"

In [32]:
def func(x):
    if x == 'Rejected':
        return 1
    elif x== 'PendingRerun':
        return 1
    else:
        return 0
    
df1['Reject_History']=df1['Receiving Status'].apply(func)

### Calculate number of rejected transactions per customer

In [33]:
count_series = df1.groupby(['Reject_History','Customer Email Address']).size()
new= count_series.to_frame(name='Reject_count').reset_index()
new=new[new['Reject_History']==1]

### Calculate dollar amount of rejected transactions per customer

In [34]:
money_value = df1[['Customer Email Address','Net Amount (Including Tip)','Reject_History']]
money_value = money_value[money_value['Reject_History']==1]
money_value.drop(['Reject_History'],axis=1,inplace=True)

In [35]:
money_value=money_value.groupby(['Customer Email Address']).sum()

In [36]:
df1=df1.merge(new,on='Customer Email Address',how='left')

In [37]:
df1=df1.merge(money_value,on='Customer Email Address',how='left')

In [38]:
df1.drop(['Reject_History_x','Reject_History_y','Receiving Status'],axis=1,inplace=True)

In [39]:
df1.head()

,Merchant Id,Merchant Name,Net Amount (Including Tip)_x,Customer User Id,Customer Email Address,Customer Phone Number,Total_Transaction_count,Total_Transaction_amount,RFM_Score,Reject_count,Net Amount (Including Tip)_y
0,1039,"Releaf Herbal Cooperative, INC",29.5200,2091,keeyasedwards@gmail.com,4154109717,2,64.2720,5,NaN,NaN
1,1039,"Releaf Herbal Cooperative, INC",27.5904,2093,ludari89@gmail.com,4157163647,28,974.8992,9,NaN,NaN
2,1039,"Releaf Herbal Cooperative, INC",54.3168,2094,jpjocelynparra@gmail.com,4159966592,2,117.0624,6,NaN,NaN
3,1039,"Releaf Herbal Cooperative, INC",44.4864,2095,yasminmahmoudian23@gmail.com,6506192470,1,44.4864,3,NaN,NaN
4,1039,"Releaf Herbal Cooperative, INC",20.6688,2096,msomach45@gmail.com,8584426873,1,20.6688,3,NaN,NaN


In [40]:
df1.columns

Index(['Merchant Id', 'Merchant Name', 'Net Amount (Including Tip)_x',
       'Customer User Id', 'Customer Email Address', 'Customer Phone Number',
       'Total_Transaction_count', 'Total_Transaction_amount', 'RFM_Score',
       'Reject_count', 'Net Amount (Including Tip)_y'],
      dtype='object')

In [41]:
df1=df1.fillna(0)
df1 = df1.drop_duplicates(subset='Customer Email Address', keep="first")
df1['Reject_count']=df1['Reject_count'].astype(int)
df1['Customer User Id']=df1['Customer User Id'].astype(int)

### Calculate the percentage of amount of rejected transaction versus the total amount of transaction

In [42]:
df1['Failed_count%'] = df1['Reject_count']/df1['Total_Transaction_count']

### Calculate the percentage of dollar volume of rejected transaction versus the total spendings

In [43]:
df1['Failed_amount%'] = df1['Net Amount (Including Tip)_y']/df1['Total_Transaction_amount']

In [44]:
df1.head()

,Merchant Id,Merchant Name,Net Amount (Including Tip)_x,Customer User Id,Customer Email Address,Customer Phone Number,Total_Transaction_count,Total_Transaction_amount,RFM_Score,Reject_count,Net Amount (Including Tip)_y,Failed_count%,Failed_amount%
0,1039,"Releaf Herbal Cooperative, INC",29.5200,2091,keeyasedwards@gmail.com,4154109717,2,64.2720,5,0,0.0,0.0,0.0
1,1039,"Releaf Herbal Cooperative, INC",27.5904,2093,ludari89@gmail.com,4157163647,28,974.8992,9,0,0.0,0.0,0.0
2,1039,"Releaf Herbal Cooperative, INC",54.3168,2094,jpjocelynparra@gmail.com,4159966592,2,117.0624,6,0,0.0,0.0,0.0
3,1039,"Releaf Herbal Cooperative, INC",44.4864,2095,yasminmahmoudian23@gmail.com,6506192470,1,44.4864,3,0,0.0,0.0,0.0
4,1039,"Releaf Herbal Cooperative, INC",20.6688,2096,msomach45@gmail.com,8584426873,1,20.6688,3,0,0.0,0.0,0.0


In [45]:
df1.columns

Index(['Merchant Id', 'Merchant Name', 'Net Amount (Including Tip)_x',
       'Customer User Id', 'Customer Email Address', 'Customer Phone Number',
       'Total_Transaction_count', 'Total_Transaction_amount', 'RFM_Score',
       'Reject_count', 'Net Amount (Including Tip)_y', 'Failed_count%',
       'Failed_amount%'],
      dtype='object')

In [46]:
df1.columns=['Merchant Id', 'Merchant Name', 'Net Amount (Including Tip)_x',
       'Customer User Id', 'Customer Email Address', 'Customer Phone Number',
       'Total_Transaction_count', 'Total_Transaction_amount', 'RFM_Score',
       'Reject_count', 'Reject_moneyvalue','Failed_count%',
       'Failed_amount%']

In [47]:
df1=df1.drop(['Net Amount (Including Tip)_x'],axis=1)

In [48]:
df1=df1.drop(['Merchant Id'],axis=1)

In [49]:
df1.head()

,Merchant Name,Customer User Id,Customer Email Address,Customer Phone Number,Total_Transaction_count,Total_Transaction_amount,RFM_Score,Reject_count,Reject_moneyvalue,Failed_count%,Failed_amount%
0,"Releaf Herbal Cooperative, INC",2091,keeyasedwards@gmail.com,4154109717,2,64.2720,5,0,0.0,0.0,0.0
1,"Releaf Herbal Cooperative, INC",2093,ludari89@gmail.com,4157163647,28,974.8992,9,0,0.0,0.0,0.0
2,"Releaf Herbal Cooperative, INC",2094,jpjocelynparra@gmail.com,4159966592,2,117.0624,6,0,0.0,0.0,0.0
3,"Releaf Herbal Cooperative, INC",2095,yasminmahmoudian23@gmail.com,6506192470,1,44.4864,3,0,0.0,0.0,0.0
4,"Releaf Herbal Cooperative, INC",2096,msomach45@gmail.com,8584426873,1,20.6688,3,0,0.0,0.0,0.0


In [51]:
df1.shape

(3200, 11)

In [50]:
df1[df1['Reject_count']>0]

,Merchant Name,Customer User Id,Customer Email Address,Customer Phone Number,Total_Transaction_count,Total_Transaction_amount,RFM_Score,Reject_count,Reject_moneyvalue,Failed_count%,Failed_amount%
136,"Universal Herbal Center, Inc d/b/a Pineapple E...",2516,twilliams62@live.com,6618749883,1,112.7043,4,1,112.7043,1.000000,1.000000
182,GoldLeaf Group,2634,jesus.sanchez101@yahoo.com,8187362564,3,214.4352,8,1,55.4784,0.333333,0.258719
261,GoldLeaf Group,2797,askworl@yahoo.com,5598040087,1,114.0384,4,1,114.0384,1.000000,1.000000
359,GoldLeaf Group,3028,amberbush622@yahoo.com,5595936505,1,128.1504,5,1,128.1504,1.000000,1.000000
361,GoldLeaf Group,3043,curtisford61@yahoo.com,5594993102,1,58.3584,3,1,58.3584,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
2696,Earth's Healing North,13726,valdezbaby730@gmail.com,5204200221,1,129.6231,7,1,129.6231,1.000000,1.000000
2698,Earth's Healing North,9821,jumjadi@gmail.com,5209904769,1,110.4115,6,1,110.4115,1.000000,1.000000
2704,Earths Healing South,13695,fixithamilton@icloud.com,5204220157,1,91.2095,6,1,91.2095,1.000000,1.000000
2745,Earth's Healing North,4874,marcovp08@gmail.com,5202714103,1,72.9676,6,1,72.9676,1.000000,1.000000
